# Model Customization Demo (Distillation)

## Model Distillation with Amazon SageMaker Training Jobs


In [1]:
#! pip install -U datasets
#!pip install --upgrade datasets
#!pip install datasets==3.6.0

In [2]:
#! pip install transformers datasets "sagemaker>=2.190.0" --upgrade --quiet
#! pip install transformers boto3 "sagemaker-core==1.0.41" "datasets==4.0.0" "sagemaker>=2.190.0" --upgrade --quiet

In [1]:
import sagemaker
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer
import boto3
import os

sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from huggingface_hub import login
# Provide hf_token value to models and data
os.environ['hf_token']=""
login(os.environ['hf_token'])


os.environ['WANDB_API_KEY'] = ""

In [3]:
from datasets import load_dataset

dataset = load_dataset('mlabonne/FineTome-100k')

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [4]:
dataset["train"] = dataset["train"].select(range(1000))

In [5]:
# save train_dataset to s3 using our SageMaker session
input_path = f's3://{sagemaker_session.default_bucket()}/datasets/distllation_training_job'

# Save datasets to s3
# We will fine tune only with 20 records due to limited compute resource for the workshop
dataset["train"].to_json(f"{input_path}/train/dataset.json", orient="records")
sft_dataset_s3_path = f"{input_path}/train/dataset.json"
# ds_train_pref["train"].to_json(f"{input_path}/pref/dataset.json", orient="records")
# perf_dataset_s3_path = f"{input_path}/pref/dataset.json"
print(f"Training data uploaded to:")
print(sft_dataset_s3_path)
print(f"https://s3.console.aws.amazon.com/s3/buckets/{sagemaker_session.default_bucket()}/?region={sagemaker_session.boto_region_name}&prefix={input_path.split('/', 3)[-1]}/")


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Training data uploaded to:
s3://sagemaker-us-east-1-783764584149/datasets/distllation_training_job/train/dataset.json
https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-783764584149/?region=us-east-1&prefix=datasets/distllation_training_job/


# ModelTrainer API

In [92]:
from sagemaker.config import load_sagemaker_config

In [93]:
configs = load_sagemaker_config()

In [134]:
from sagemaker.modules.train import ModelTrainer
from sagemaker.modules.configs import Compute, SourceCode, InputData, StoppingCondition, CheckpointConfig

env = {}
env["FI_PROVIDER"] = "efa"
env["NCCL_PROTO"] = "simple"
env["NCCL_SOCKET_IFNAME"] = "eth0"
env["NCCL_IB_DISABLE"] = "1"
env["NCCL_DEBUG"] = "WARN"
env["HF_token"] = os.environ['hf_token']
#env["WANDB_API_KEY"] = os.environ['WANDB_API_KEY']
env["data_location"] = sft_dataset_s3_path
# MLFlow tracker
tracking_server_arn = 'arn:aws:sagemaker:us-east-1:783764584149:mlflow-tracking-server/MLflow3-test'
mlflow_experiment_name = 'distillation'
env["MLFLOW_TRACKING_ARN"] = tracking_server_arn
env["MLFLOW_EXPERIMENT_NAME"] = mlflow_experiment_name
instance_type= "ml.g6.48xlarge" # "ml.p4de.24xlarge"
compute = Compute(
    instance_count=1,
    instance_type= instance_type,
    volume_size_in_gb=500,
    keep_alive_period_in_seconds=3600,
)

In [135]:
# image_uri = (
#     f"658645717510.dkr.ecr.{sagemaker_session.boto_session.region_name}.amazonaws.com/smdistributed-modelparallel:2.4.1-gpu-py311-cu121"
# )

# image_uri

image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sagemaker_session.boto_session.region_name,
    version="2.6.0",
    instance_type=instance_type,
    image_scope="training"
)
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312'

In [136]:
checkpoint_s3_path = f"s3://{bucket_name}/distillation-checkpoints/checkpoints"
checkpoint_s3_path

's3://sagemaker-us-east-1-783764584149/distillation-checkpoints/checkpoints'

In [137]:
job_prefix = f"model-distillation-Qwen3-1-7B-0-6B"

In [138]:
hyperparameters = {
    "dataset_path": "/opt/ml/input/data/dataset",
    "model_dir": "/opt/ml/model",
    "MLFLOW_TRACKING_ARN": tracking_server_arn,
    "MLFLOW_EXPERIMENT_NAME": mlflow_experiment_name
}

In [139]:
source_code = SourceCode(
    source_dir="./scripts",
    requirements="requirements.txt",
    entry_script="run_kd.sh",
)

In [140]:
model_trainer = ModelTrainer(
    training_image=image_uri,
    compute=compute,
    hyperparameters=hyperparameters,
    environment=env,
    source_code=source_code,
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=90000,
    ),
    checkpoint_config=CheckpointConfig(
        s3_uri=f"{checkpoint_s3_path}/{job_prefix}",
    ),
    base_job_name=job_prefix
)

[10/14/25 22:32:34] WARNING  SageMaker session not provided. Using default Session.            ]8;id=941333;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=646631;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#501\501]8;;\

                    WARNING  Role not provided. Using default role:                            ]8;id=391849;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=852932;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#505\505]8;;\
                             arn:aws:iam::783764584149:role/service-role/AmazonSageMaker-Execu                     
                             tionRole-20241230T144802                                                              

[10/14/25 22:32:35] WARNING  OutputDataConfig not provided. Using default:                     ]8;id=90435;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=316325;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#567\567]8;;\
                             s3_output_path='s3://sagemaker-us-east-1-783764584149/model-disti                     
                             llation-Qwen3-1-7B-0-6B' kms_key_id=None compression_type='GZIP'                      

                    INFO     Training image URI:                                               ]8;id=15930;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=733161;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#588\588]8;;\
                             763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6                     
                             .0-gpu-py312                                                                          

In [141]:
sft_dataset_s3_path

's3://sagemaker-us-east-1-783764584149/datasets/distllation_training_job/train/dataset.json'

In [142]:
training_data = InputData(
    channel_name="training_dataset",
    data_source=sft_dataset_s3_path,
)

In [143]:
model_trainer.train(input_data_config=[training_data], wait=False)

                    WARNING  key_prefix is only applicable when data_source is a local file    ]8;id=775236;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=279554;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#896\896]8;;\
                             path.                                                                                 

[10/14/25 22:32:36] INFO     Creating training_job resource.                                     ]8;id=392994;file:///opt/conda/lib/python3.12/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=9466;file:///opt/conda/lib/python3.12/site-packages/sagemaker_core/main/resources.py#28522\28522]8;;\

[10/14/25 22:32:36] WARNING  Not displaing the training container logs as 'wait' is set to     ]8;id=450857;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=278857;file:///opt/conda/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#834\834]8;;\
                             False.                                                                                

In [ ]:
def get_last_job_name(job_name_prefix):
    sagemaker_client = boto3.client('sagemaker')

    matching_jobs = []
    next_token = None

    while True:
        # Prepare the search parameters
        search_params = {
            'Resource': 'TrainingJob',
            'SearchExpression': {
                'Filters': [
                    {
                        'Name': 'TrainingJobName',
                        'Operator': 'Contains',
                        'Value': job_name_prefix
                    },
                    {
                        'Name': 'TrainingJobStatus',
                        'Operator': 'Equals',
                        'Value': "Completed"
                    }
                ]
            },
            'SortBy': 'CreationTime',
            'SortOrder': 'Descending',
            'MaxResults': 100
        }

        # Add NextToken if we have one
        if next_token:
            search_params['NextToken'] = next_token

        # Make the search request
        search_response = sagemaker_client.search(**search_params)

        # Filter and add matching jobs
        matching_jobs.extend([
            job['TrainingJob']['TrainingJobName'] 
            for job in search_response['Results']
            if job['TrainingJob']['TrainingJobName'].startswith(job_name_prefix)
        ])

        # Check if we have more results to fetch
        next_token = search_response.get('NextToken')
        if not next_token or matching_jobs:  # Stop if we found at least one match or no more results
            break

    if not matching_jobs:
        raise ValueError(f"No completed training jobs found starting with prefix '{job_name_prefix}'")

    return matching_jobs[0]

In [ ]:
#job_prefix = 

In [ ]:
job_name = get_last_job_name(job_prefix)

job_name

In [ ]:
if default_prefix:
    model_data=f"s3://{bucket_name}/{default_prefix}/{job_prefix}/{job_name}/output/model.tar.gz"
else:
    model_data=f"s3://{bucket_name}/{job_prefix}/{job_name}/output/model.tar.gz"


In [ ]:
model_data